# Converting between data loaded from GeoTIFFs and ODC

* Add/remove CRS (`spatial_ref`) metadata
* Collapse or extract `data_variable` elements

### Load packages

In [1]:
%matplotlib inline
import datacube
import xarray as xr
import matplotlib.pyplot as plt

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb

# add metadata to arrays
from datacube.utils.geometry import assign_crs
from odc.algo import xr_reproject

/env/lib/python3.8/site-packages/datacube/storage/masking.py:7: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  warnings.warn("datacube.storage.masking has moved to datacube.utils.masking",


### Connect to the datacube

In [2]:
dc = datacube.Datacube(app='Converting_data')

## ODC to TIFF-style (xarray.Dataset to xarray.DataArray)

Using `load_ard` or `dc.load` imports datasets as `xarray.Dataset` objects with metadata such as CRS assigned as Coordinates during the loading process.

TIFFs imported using rasterio are turned into DataArrays and multiple bands of data are concatenated as a multi-dimensional array.

### Load data - from datacube

In [3]:
# Set the central latitude and longitude
central_lat = -1.286389
central_lon = 36.817223
buffer = 0.05

# Compute the bounding box for the study area
lat_range = (central_lat-buffer, central_lat+buffer)
lon_range = (central_lon-buffer, central_lon+buffer)

# time frame for the analysis 
time_range = ('2020-03-11')

In [4]:
# View the study area 
# display_map(x=lon_range, y=lat_range)

In [5]:
ds_odc = load_ard(dc=dc,
              products=["s1_rtc"],
              measurements=['vv', 'vh', 'mask'],
              y=lat_range,
              x=lon_range,
              time=time_range,
              output_crs = "EPSG:6933",
              resolution = (-10,10),
              group_by="solar_day"
             ).squeeze()

Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 1 time steps


CPLReleaseMutex: Error = 1 (Operation not permitted)


In [6]:
ds_odc

<xarray.Dataset>
Dimensions:      (y: 1277, x: 966)
Coordinates:
    time         datetime64[ns] 2020-03-11T15:48:15.345725
  * y            (y) float64 -1.577e+05 -1.577e+05 ... -1.705e+05 -1.705e+05
  * x            (x) float64 3.548e+06 3.548e+06 ... 3.557e+06 3.557e+06
    spatial_ref  int32 6933
Data variables:
    vv           (y, x) float32 0.1992 0.1992 0.1367 ... 0.3466 0.3466 1.046
    vh           (y, x) float32 0.03954 0.03954 0.04905 ... 0.2316 0.2316 0.6343
    mask         (y, x) uint8 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

`ds_odc` loads natively as an xarray.Dataset with the measurements specified as `Data variables` - here, `vv`, `vh` and `mask`. 

In [7]:
ds_odc.to_array()

<xarray.DataArray (variable: 3, y: 1277, x: 966)>
array([[[0.19918314, 0.19918314, 0.13665111, ..., 0.19162583,
         0.19162583, 0.19019802],
        [0.19057332, 0.19057332, 0.14621727, ..., 0.19807614,
         0.19807614, 0.37474817],
        [0.19057332, 0.19057332, 0.14621727, ..., 0.19807614,
         0.19807614, 0.37474817],
        ...,
        [0.10714412, 0.10714412, 0.13690189, ..., 0.22719534,
         0.22719534, 0.69750464],
        [0.10714412, 0.10714412, 0.13690189, ..., 0.22719534,
         0.22719534, 0.69750464],
        [0.2006324 , 0.2006324 , 0.10588653, ..., 0.34661674,
         0.34661674, 1.0460314 ]],

       [[0.03954393, 0.03954393, 0.04905235, ..., 0.0356207 ,
         0.0356207 , 0.04137892],
        [0.04230079, 0.04230079, 0.05054975, ..., 0.04774575,
         0.04774575, 0.06185948],
        [0.04230079, 0.04230079, 0.05054975, ..., 0.04774575,
         0.04774575, 0.06185948],
...
        [0.04742814, 0.04742814, 0.04369544, ..., 0.12448375,
         0.12448375, 0.44420743],
        [0.04742814, 0.04742814, 0.04369544, ..., 0.12448375,
         0.12448375, 0.44420743],
        [0.08270667, 0.08270667, 0.0501078 , ..., 0.23162667,
         0.23162667, 0.6342905 ]],

       [[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ]]], dtype=float32)
Coordinates:
    time         datetime64[ns] 2020-03-11T15:48:15.345725
  * y            (y) float64 -1.577e+05 -1.577e+05 ... -1.705e+05 -1.705e+05
  * x            (x) float64 3.548e+06 3.548e+06 ... 3.557e+06 3.557e+06
    spatial_ref  int32 6933
  * variable     (variable) <U4 'vv' 'vh' 'mask'
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

## TIFF to ODC-style (xarray.DataArray to xarray.Dataset)

1. Import tiff using `xr.open_rasterio` and inspect. Do not `squeeze()`.
2. Explicitly assign a CRS. This can be done manually by using  `datacube.utils.geometry.assign_crs`. It can also be done automatically using `xr_reproject` to steal a CRS from your other datasets.
3. Drop Attribute metadata (such as crs attributes) that are no longer accurate.
4. Convert DataArray to a DataSet
5. Rename `Data variable` to desired band names.


### Importing a TIFF file

In [8]:
# Load the Esri Land cover 2020 global land use/land cover (LULC) product
# It has one variable, band, which contains values from 0 to 7

tif = './ESRI2020Landcover.tiff'
ds_tiff = xr.open_rasterio(tif) # do not squeeze - keeps band as a variable

In [9]:
ds_tiff

<xarray.DataArray (band: 1, y: 1108, x: 1115)>
[1235420 values with dtype=uint8]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 9.863e+06 9.863e+06 9.863e+06 ... 9.852e+06 9.852e+06
  * x        (x) float64 2.515e+05 2.516e+05 2.516e+05 ... 2.627e+05 2.627e+05
Attributes:
    transform:      (10.0, 0.0, 251540.0, 0.0, -10.0, 9863250.0)
    crs:            +init=epsg:32737
    res:            (10.0, 10.0)
    is_tiled:       0
    nodatavals:     (0.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area

We need to assign CRS as one of the coordinates. `We can do this using datacube.utils.geometry.assign_crs`.

In [10]:
ds_tiff = datacube.utils.geometry.assign_crs(ds_tiff, crs= 'EPSG:6933')

We could also have added spatial_ref to Coordinates by using geobox, which retrieves the CRS from an existing dataset. 
For example:
```
ds_tiff_with_crs = datacube.utils.geometry.assign_crs(ds_tiff, crs=odc_ds.geobox.crs)
```

Note the `crs:` attribute does not have any significance - it has been pulled from the tif file but is not a coordinate of the xarray. We can drop it now it does not match the actual `spatial_ref` coordinate.

In [11]:
del ds_tiff.attrs['crs']

In [12]:
ds_tiff

<xarray.DataArray (band: 1, y: 1108, x: 1115)>
[1235420 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * y            (y) float64 9.863e+06 9.863e+06 ... 9.852e+06 9.852e+06
  * x            (x) float64 2.515e+05 2.516e+05 ... 2.627e+05 2.627e+05
    spatial_ref  int32 6933
Attributes:
    transform:      (10.0, 0.0, 251540.0, 0.0, -10.0, 9863250.0)
    res:            (10.0, 10.0)
    is_tiled:       0
    nodatavals:     (0.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    grid_mapping:   spatial_ref

Now we can transform this to a Dataset and use the band coordinate as our data variable.

In [13]:
ds_tiff = ds_tiff.to_dataset(dim='band')

Finally we can rename the data variable to match what we want. In this example we use "band".

In [14]:
ds_tiff = ds_tiff.rename(name_dict={1:'band'})

In [15]:
ds_tiff

<xarray.Dataset>
Dimensions:      (y: 1108, x: 1115)
Coordinates:
  * y            (y) float64 9.863e+06 9.863e+06 ... 9.852e+06 9.852e+06
  * x            (x) float64 2.515e+05 2.516e+05 ... 2.627e+05 2.627e+05
    spatial_ref  int32 6933
Data variables:
    band         (y, x) uint8 ...
Attributes:
    transform:      (10.0, 0.0, 251540.0, 0.0, -10.0, 9863250.0)
    res:            (10.0, 10.0)
    is_tiled:       0
    nodatavals:     (0.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    grid_mapping:   spatial_ref

The band can now be called:

In [16]:
ds_tiff.band

<xarray.DataArray 'band' (y: 1108, x: 1115)>
[1235420 values with dtype=uint8]
Coordinates:
  * y            (y) float64 9.863e+06 9.863e+06 ... 9.852e+06 9.852e+06
  * x            (x) float64 2.515e+05 2.516e+05 ... 2.627e+05 2.627e+05
    spatial_ref  int32 6933